<a href="https://colab.research.google.com/github/dinh-thang/COS30018-Project-C/blob/main/Falcon_1b_with_Biomed_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers einops
!pip install git+https://github.com/huggingface/accelerate
!pip install transformers[torch]
!pip install git+https://github.com/huggingface/peft.git
!pip install datasets bitsandbytes wandb trl
!pip install --upgrade transformers
!pip install evaluate numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.7 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-q07d7_c1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-q07d7_c1
  Resolved https://github.com/huggingface/accelerate to commit 8f45a2eae8fe6325a16ae0e13c3653652950f4cd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.24.0.dev0-py3-no

# Load the dataset

In [ ]:
from datasets import load_dataset

dataset_name = "covid_qa_deepset"
dataset = load_dataset(dataset_name)
dataset = dataset["train"].train_test_split(test_size=20, train_size=1413)
dataset = dataset.filter(lambda example: not "answer_category" in example['answers'])

Generating train split:   0%|          | 0/2019 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1413 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['document_id', 'context', 'question', 'is_impossible', 'id', 'answers'],
        num_rows: 1413
    })
    test: Dataset({
        features: ['document_id', 'context', 'question', 'is_impossible', 'id', 'answers'],
        num_rows: 20
    })
})

In [ ]:
import ast
import torch
from transformers import AutoTokenizer


# load falcon-7b tokenizer
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token


def merge_answer(record):
  answer = record['answers']['text'][0]
  record['answer'] = answer
  return record

def add_input_ids(record):
  question = record["question"]
  context = record["context"]
  inputs = tokenizer(
      question,
      context,
      truncation="only_second",
      max_length=512,
      padding="max_length",
  )
  record['input_ids'] = inputs['input_ids']
  record['attention_mask'] = inputs['attention_mask']
  return record

def add_answer_start(record):
  record['start_positions'] = record['answers']['answer_start'][0]
  return record

def add_answer_end(record):
  record['end_positions'] = record['start_positions'] + len(record['answer'])
  return record

training_dataset = dataset["train"]
eval_dataset = dataset["test"]

training_dataset = training_dataset.add_column(name="input_ids", column=[0]*len(training_dataset))
training_dataset = training_dataset.add_column(name="attention_mask", column=[0]*len(training_dataset))
training_dataset = training_dataset.add_column(name="answer", column=[0]*len(training_dataset))
training_dataset = training_dataset.add_column(name="start_positions", column=[0]*len(training_dataset))
training_dataset = training_dataset.add_column(name="end_positions", column=[0]*len(training_dataset))


training_dataset = training_dataset.map(merge_answer)
training_dataset = training_dataset.map(add_input_ids)
training_dataset = training_dataset.map(add_answer_start)
training_dataset = training_dataset.map(add_answer_end)
training_dataset = training_dataset.remove_columns(column_names=['answers','context', 'question','id','document_id', 'is_impossible'])


eval_dataset = eval_dataset.add_column(name="input_ids", column=[0]*len(eval_dataset))
eval_dataset = eval_dataset.add_column(name="attention_mask", column=[0]*len(eval_dataset))
eval_dataset = eval_dataset.add_column(name="answer", column=[0]*len(eval_dataset))
eval_dataset = eval_dataset.add_column(name="start_positions", column=[0]*len(eval_dataset))
eval_dataset = eval_dataset.add_column(name="end_positions", column=[0]*len(eval_dataset))

eval_dataset = eval_dataset.map(merge_answer)
eval_dataset = eval_dataset.map(add_input_ids)
eval_dataset = eval_dataset.map(add_answer_start)
eval_dataset = eval_dataset.map(add_answer_end)
eval_dataset = eval_dataset.remove_columns(column_names=['answers','context', 'question','id','document_id', 'is_impossible'])




Flattening the indices:   0%|          | 0/1413 [00:00<?, ? examples/s]

Map:   0%|          | 0/1413 [00:00<?, ? examples/s]

Map:   0%|          | 0/1413 [00:00<?, ? examples/s]

Map:   0%|          | 0/1413 [00:00<?, ? examples/s]

Map:   0%|          | 0/1413 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [ ]:
training_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'answer', 'start_positions', 'end_positions'],
    num_rows: 1413
})

In [ ]:
eval_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'answer', 'start_positions', 'end_positions'],
    num_rows: 20
})

In [ ]:
training_dataset['answer']


['due to the complexity and heterogeneity of the disease.',
 'calculate the relatedness of objects with the same or different types',
 '2 strains out of 25 (8%)',
 'seropositive and RNA or virus negative',
 'single-strand RNA filoviruses',
 'smokers were 1.4 times more likely (RR=1.4, 95% CI: 0.98–2.00) to have severe symptoms of COVID-19 and approximately 2.4 times more likely to be admitted to an ICU, need mechanical ventilation or die compared to non-smokers (RR=2.4, 95% CI: 1.43–4.04)',
 'the antiproliferative effect of a copper (II) complex on HT-29 colon cancer cells',
 'Clinical studies have started to correlate specific biomarkers with disease outcomes in ARDS patients',
 'RNAse P, a marker of human cells,',
 'Global alignment of case definitions',
 'the microbe occurs in no other disease as a fortuitous and nonpathogenic parasite',
 'the transmissibility of SARS-CoV-2 might be higher than MERS in the Middle East countries, similar to SARS, but lower than MERS in the Republic o

In [ ]:
eval_dataset['input_ids'][0]


[2205,
 441,
 13930,
 11576,
 2767,
 1952,
 42,
 44,
 289,
 10475,
 275,
 47115,
 52708,
 301,
 891,
 23545,
 482,
 272,
 7735,
 1649,
 7838,
 272,
 35527,
 193,
 193,
 12659,
 7561,
 17794,
 25,
 17747,
 6825,
 25,
 24084,
 88,
 25,
 6298,
 83,
 25,
 16162,
 26,
 4111,
 78,
 26,
 34067,
 26,
 7141,
 46,
 43721,
 24445,
 27,
 26,
 193,
 193,
 40853,
 37,
 293,
 20424,
 2102,
 469,
 27,
 310,
 3519,
 2034,
 42267,
 38579,
 47621,
 9188,
 8199,
 1704,
 77,
 32,
 6869,
 2423,
 81,
 31,
 7427,
 36,
 193,
 193,
 43382,
 37,
 26611,
 23,
 2641,
 38,
 6796,
 316,
 5165,
 254,
 23,
 1781,
 37554,
 38,
 8372,
 15096,
 363,
 23,
 30882,
 38,
 531,
 3084,
 1910,
 23,
 29907,
 38,
 531,
 7449,
 254,
 23,
 12948,
 38,
 14394,
 468,
 330,
 254,
 23,
 14037,
 12985,
 38,
 7937,
 84,
 254,
 23,
 2346,
 25796,
 38,
 1781,
 349,
 5900,
 254,
 23,
 30882,
 193,
 7499,
 37,
 204,
 626,
 33,
 24,
 3091,
 24,
 1313,
 193,
 40144,
 37,
 204,
 696,
 25,
 19469,
 28,
 26,
 35048,
 25,
 45707,
 25,
 31960,
 397

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
training_dataset.push_to_hub("goodcoffee/covidQA_training")
eval_dataset.push_to_hub("goodcoffee/covidQA_eval")


OSError: ignored

# Load the model

In [ ]:
import torch
import transformers
from transformers import AutoModelForQuestionAnswering, BitsAndBytesConfig, FalconForQuestionAnswering, AutoConfig

model_name = "hung200504/falcon-7b-sharded-bf16-finetuned"
base_model_name = "tiiuae/falcon-7b"

config = AutoConfig.from_pretrained(base_model_name, trust_remote_code=True, max_new_tokens=2048)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForQuestionAnswering.from_pretrained(
    model_name,
    config=config,
    quantization_config=bnb_config,
    trust_remote_code=True,
    torch_dtype=torch.float32
)
model.config.use_cache = False


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

# Configuration for QLoRa

In [ ]:
from peft import LoraConfig

lora_alpha = 16 # based on qlora paper
lora_dropout = 0.1 # for model up to 13B
lora_r = 64 # based on qlora paper

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="QUESTION_ANSWERING",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)


# Load evaluation metrics

In [ ]:
import evaluate
import numpy as np

metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions[0], references=labels[0],average="macro")

In [ ]:
metric.description

In [ ]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 1
gradient_accumulation_steps = 1
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3 # based on qlora paper
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    report_to=None
)


In [ ]:
from transformers import Trainer
from peft import PeftModelForQuestionAnswering

max_seq_length = 2048
peft_model = PeftModelForQuestionAnswering(model=model, peft_config=peft_config)
peft_model.to(torch.device("cuda"))

trainer = Trainer(
    model=peft_model,
    train_dataset=training_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    args=training_arguments,
    compute_metrics=compute_metrics,
)

In [ ]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login(key="2ba90e109c0da4331467290174e54e3483f0d494")
wandb.init(project="QLoRA covidQA")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bobbyngoson204 (intelligentsystem). Use `wandb login --relogin` to force relogin


In [ ]:
trainer.train()


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,0.000000
20,0.000000
30,0.000000
40,2.423400
50,0.000000
60,0.000000
70,0.000000
80,0.653100
90,0.871200
100,0.000000


Step,Training Loss
10,0.000000
20,0.000000
30,0.000000
40,2.423400
50,0.000000
60,0.000000
70,0.000000
80,0.653100
90,0.871200
100,0.000000


TrainOutput(global_step=500, training_loss=0.19368461418151856, metrics={'train_runtime': 973.5481, 'train_samples_per_second': 0.514, 'train_steps_per_second': 0.514, 'total_flos': 1.037847174144e+16, 'train_loss': 0.19368461418151856, 'epoch': 0.35})

#Run evaluation

In [ ]:
trainer.evaluate()

{'eval_loss': nan,
 'eval_f1': 0.0,
 'eval_runtime': 11.1699,
 'eval_samples_per_second': 1.791,
 'eval_steps_per_second': 0.269,
 'epoch': 0.35}

In [ ]:
import torch
torch.cuda.empty_cache()


# Inference

In [ ]:
from transformers import QuestionAnsweringPipeline
question = dataset['train']['question'][0]
context = dataset['train']['context'][0]
print(question)
#print(context)

pipeline = QuestionAnsweringPipeline(model=model, tokenizer=tokenizer)
print(pipeline(question=question, context=context, max_answer_len=300, max_question_len=300))

Why do treatment and management vary in efficacy?
{'score': 2.0400439098011702e-05, 'start': 856, 'end': 1069, 'answer': ' vitro air-liquid interface 3D cultures, organoid cultures and the use of novel human and animal challenge models have evoked new understandings as to the mechanisms of viral exacerbations. In this review, we will'}
